# Flamingo

## 参考

- [Flamingo: a Visual Language Model for Few-Shot Learning][0]
- [OpenFlamingo: An Open-SOurce Framework for Training Large Autogressive Vision-Language Models][1]
- [mlfoundations / open_flamingo][2]

[0]: https://arxiv.org/abs/2204.14198
[1]: https://arxiv.org/abs/2308.01390
[2]: https://github.com/mlfoundations/open_flamingo

## 概要

Flamingoは、少数のアノテーションサンプルで新しいタスクに迅速に対応できる視覚言語モデル（Visual Language Model, VLM）

事前学習済みの視覚モデルと言語モデルを連携し、画像・動画とテキストの入力に対応したアーキテクチャを採用

インターネットから収集した大規模なテキストと画像が混在するコーパスで訓練

コンテキストに回答例を含めて新しいタスクに柔軟に対応する学習を実現（in-context few-shot learning）

訓練したモデルを詳細に評価:

1. 画像に関する質問に回答するオープンエンドなタスク（Visual Question-Answering, VQA）
2. 画像や動画にキャプションをつけるタスク
3. 画像に関する多肢選択式の質問に回答するクローズドエンドなタスク

多くのベンチマークで、ファインチューニング済みの他のモデルよりも高い性能

Flaming-80Bでのフューショットプロンプトに対する出力サンプル:

![](image/fig1_1.png)

複数の画像にわたる対話も可能:

![](image/fig1_2.png)

Flamingoが16のうち6つのタスクで最先端の特化モデルよりも優れていて、モデルサイズと例示数が増えると性能が上がる:

![](image/fig2.png)

## アーキテクチャ

Flamingoは、2つのパラメータを固定した（frozen）モデルを活用:

1. 視覚モデル（Vision Encoder）: 画像や動画を知覚する
2. 言語モデル（LM block）: 基本的な推論を行う

事前学習中に獲得した知識を保持するコンポーネント:

- Perceiver Resampler: 可変な高解像度の画像や動画を、少数の固定された視覚トークンに変換する
- ゲート付きクロスアテンション密結合（GATED XATTN-DENSE）: 視覚モデルと言語モデルを橋渡しする

![](image/fig3.png)

Flamingoは、生成するテキスト全体 $y$ の尤度を次式でモデル化:

$$
p(y|x) = \prod_{l=1}^L p(y_l | y_{\le l}, x_{\le l})
$$

- テキスト全体の生成確率 $p(y|x)$ は、今までに見た全ての視覚トークン $x_{\le l}$ と今までに生成したテキストトークン $y_{\le l}$ の条件下で次のトークン $y_l$ が生成される確率を、最初から最後までかけ合わせた値
- $y_l$: テキスト $y$ の $l$ 番目の言語トークン
- $y_{\le l}$: $l$ 番目のトークンより前にある全ての言語トークン
- $x_{\le l}$: $l$ 番目のトークンが処理されるより前のすべての画像や動画の集合
- $p(\cdot)$: モデルによって推論される確率

視覚エンコーダーの特徴:

- Normalizer-Free ResNet（NFNet）を使用
- CLIPと同様の対象学習損失（two-term contrastive loss）で事前学習
- 視覚エンコーダーの出力は1次元にフラット化される
- 動画入力の場合、1FPSでサンプリングし、独立してエンコードし、学習済みの時間埋め込みを加算しフラット化
- NFNetエンコーダーのようなCNNの場合、暗黙的に空間情報をチャンネルごとに含んでいるため画像の位置埋め込みは必要ない

Perceiver Resamplerの特徴:

- 視覚エンコーダーからの可変長な入力を、固定長（=64トークン）の視覚トークン（latent input queries）に変換
- Perceiver Resamplerは、学習可能な潜在クエリ（=64トークン）を入力としたクロスアテンションを計算する複数のTransformerで構成される

![](image/fig5.png)

ゲート付きクロスアテンド密結合（GATED XATTN-DENSE）の特徴：

- ゲート付きのクロスアテンション層とゲート付きフィードフォワード層で構成される
- 事前学習済みの言語モデルブロックを凍結し、元の層の間にGATED XATTN-DENSEを追加し、ゼロから訓練する
- ゲートはtanh-gating（ハイパボリックタンジェントゲーティング）を採用し、ゼロで初期化
- 訓練の最初は元の言語モデルと変わらないため学習が安定化し、その結果最終的な結果も改善

![](image/fig4.png)

言語モデルはChinchillaモデルを使用し、3つのサイズで実験:

- 1.4B（Flamingo-3B）
- 7B（Flamingo-9B）
- 70B（Flamingo-80B）

画像因果モデリング（image-causal modeling）の仕組み:

- 画像因果モデリングは、今まで出現したテキストと画像でトークンの生成が条件付けられる性質
- クロスアテンション行列にマスキングし、アテンドできる視覚トークンを「直前に出現した画像のみ」に制限することで実現
- この単一画像クロスアテンション（single-image cross attention）により、複数枚画像入力に汎化可能になる

訓練は3種類のデータセットの混ぜ合わせて実施:

1. 画像とテキストが交互に配置されたサンプルで構成されるデータセット
    - 4300万件のウェブページから抽出し、HTML構造をもとに再構成したM3W（MultiModal Massive Web）を作成
    - 画像の位置に`<image>`タグを追加し、各画像の直前とドキュメント末尾に`<EOC>`（end of chunk）トークンを追加
    - ドキュメントごとに $L=256$ トークンをランダムにサンプリングし、$N = 5$ 枚までの画像を使用
2. 画像とテキストのペアで構成されるデータセット
    - 18億の画像とaltテキストがペアになったALIGNデータセットをベースに使用
    - 更により高品質で長い説明文を含む3億1200万件のデータセットを作成し補完（LTIP, Long Text & Image Pairs）
3. 動画とテキストのペアで構成されるデータセット
    - 平均約22秒の短い動画と説明文がペアになった2700万件のデータで構成（VTP, Video & Text Pairs）

訓練の損失関数は、データセットごとに重み付けされた期待負対数尤度（expected negative log-liklihood）を使用:

$$
\sum_{m=1}^M \lambda_m \cdot \mathbb{E}_{(x,y) \sim D_m} \left[ - \sum_{l=1}^L \log p(y_l | y_{\lt l}, x_{\le l}) \right]
$$

- 各データセット（$D_m$）について、負の対数尤度の平均（$\mathbb{E}$）を計算し、各データセットの重要度 $\lambda_m$ で重み付けし合計した値
- $D_m$: $m$ 番目のデータセット
- $\lambda_m$: $m$ 番目のデータセットの重要度

訓練では、全てのデータセットからバッチを計算し、それぞれの勾配を個別に計算し、合算してパラメータを更新する（多目的訓練）

訓練後は、インコンテキスト学習（in-context learning）を用いてモデルのタスク適応力を評価:

- 画像とテキストがペアになった回答例（support example pairs）の後に、クエリとなる画像を追加してプロンプトを構成
- オープンエンドの評価では、ビームサーチを使用
- クローズドエンドの評価では、モデルの対数尤度を用いて選択肢をスコアリング

## 実験

11のベンチマークで性能を検証し、そのうち7つが画像と動画認識タスクで特化モデルより優れた性能を示した:

![](image/table1.png)

FlamingoがSotAに達しなかった13のベンチマークは、入力解像度を上げるファインチューニングをすると5つが改善:

![](image/table2.png)

Flamingo-3Bでアブレーション研究をしたところ訓練データの混合（M3W）が特に重要であることがわかった:

![](image/table3.png)

## 実装

[Open-Flamingo][1]を参考にして実装する。

[1]: https://github.com/mlfoundations/open_flamingo

### 環境構築

In [ ]:
# ログ設定

import logging
import os

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)
logger.info("ログを初期化")

In [ ]:
%pip install -q open_clip_torch einops einops_exts

In [ ]:
from einops import rearrange
from einops import repeat
from einops_exts import rearrange_many
from huggingface_hub import hf_hub_download
from PIL import Image
from torch import einsum, nn
from torch import nn
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp.wrap import enable_wrap, wrap
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast
from typing import Optional
import open_clip
import requests
import torch
import torch.nn as nn

### ヘルパー関数

In [ ]:
def exists(val):
    """
    値がNoneでないかを検証する
    モデルの設定のオプションでNoneを検証する際のショートカット

    引数:
        val: 検証する値
    戻り値:
        bool: 値がNoneでない場合にTrueを返す
    """
    return val is not None

exists("test")

In [ ]:
def getattr_recursive(obj, att):
    """
    オブジェクトのネストされた属性を属性名で取得する
    既存の言語モデルにFlamingoをミックスインする際に役立つ

    引数:
        obj: 属性を取得するオブジェクト
        att (str): ネストされた属性名（ドットで区切られた文字列）
    戻り値:
        属性の値
    """
    if att == "":
        return obj
    i = att.find(".")
    if i < 0:
        return getattr(obj, att)
    else:
        return getattr_recursive(getattr(obj, att[:i]), att[i + 1 :])

# 検証

class Nested:
    b = None

class Sample:
    a = Nested()

sample = Sample()
sample.a.b = 42
getattr_recursive(sample, "a.b")

In [ ]:
def setattr_recursive(obj, att, val):
    """
    オブジェクトにネストされた属性を属性名で設定する
    既存の言語モデルにFlamingoをミックスインする際に役立つ

    引数:
        obj: 属性を設定するオブジェクト
        att (str): ネストされた属性名（ドットで区切られた文字列）
        val: 設定する値
    """
    if "." in att:
        obj = getattr_recursive(obj, ".".join(att.split(".")[:-1]))
    setattr(obj, att.split(".")[-1], val)

# 検証

sample = Sample()
setattr_recursive(sample, "a.b", 42)
sample.a.b

### FeedFoward

In [ ]:
def FeedForward(dim, mult=4):
    """
    フィードフォワードニューラルネットワークを作成する

    2箇所で使用する
    1. PerceiverResampler
    2. GatedCrossAttention内のMaskedCrossAttentionで使用する

    引数:
        dim (int): 入力および出力の次元数
        mult (int, optional): 内部層の次元数を決定するための乗数。デフォルトは4。
    戻り値:
        nn.Sequential: フィードフォワードニューラルネットワークのシーケンシャルモデル
    """
    logger.info(f"フィードフォワードネットワークを初期化 {dim=} {mult=}")

    # PerceiverResamplerの場合 1024 * 4 = 4096
    # MaskedCrossAttentionの場合 2048 * 4 = 8192
    inner_dim = int(dim * mult)

    res = nn.Sequential(
        nn.LayerNorm(dim), # 1024 or 2048
        nn.Linear(dim, inner_dim, bias=False), # 1024 -> 4,096 or 2048 -> 8,192
        nn.GELU(), # Gaussian Error Linear Unit活性化関数
        nn.Linear(inner_dim, dim, bias=False), # 4,096 -> 1,024 or 8,192 -> 2,048
    )

    logger.info("フィードフォワードネットワークを初期化完了")
    return res

ff = FeedForward(128, mult=2)
ff

### MaskedCrossAttention

In [ ]:
class MaskedCrossAttention(nn.Module):
    """
    マスク付きクロスアテンションモジュール
    論文中のGATED XATTN-DENSEのcross attentionブロックの実装
    GatedCrossAttentionBlockで使用
    """

    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        only_attend_immediate_media=True,
    ):
        logger.info(f"MaskedCrossAttentionを初期化開始 {dim=}, {dim_visual=}, {dim_head=}, {heads=}, {only_attend_immediate_media=}")

        super().__init__()

        # QKのスケーリング値
        # 64^-0.5 = 0.125
        self.scale = dim_head**-0.5
        logger.debug(f"{self.scale=}")

        # アテンションヘッド数
        # 8
        self.heads = heads

        # アテンション計算の内部次元
        # 64 * 8 = 512
        inner_dim = dim_head * heads
        logger.info(f"{inner_dim=}")

        # アテンション計算前のレイヤー正規化層
        # 2048
        self.norm = nn.LayerNorm(dim)

        # クエリへの線形射影層
        # テキスト特徴量が入力される
        # 2048 -> 512
        self.to_q = nn.Linear(dim, inner_dim, bias=False)

        # キーとバリューへの線形射影層
        # 視覚特徴量が入力される
        # 1024 -> 1024
        self.to_kv = nn.Linear(dim_visual, inner_dim * 2, bias=False)

        # アテンション出力への線形射影層
        # 512 -> 2048
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

        # 直前の画像のみに注意を向けるか、すべての前の画像に注意を向けるか
        # True
        self.only_attend_immediate_media = only_attend_immediate_media

        logger.info("MaskedCrossAttentionの初期化完了")


    def forward(self, x, media, media_locations=None, use_cached_media=False):
        """
        マスク付きクロスアテンションの順伝播
        画像特徴量をキーとバリュー、テキスト特徴量をクエリとして使用する

        引数:
            x (torch.Tensor): テキスト特徴量
                shape (B, T_txt, D_txt)
            media (torch.Tensor): 画像特徴量
                shape (B, T_img, n, D_img) ここでnは潜在変数の次元
            media_locations: x内のメディアトークンを識別するブールマスク
                shape (B, T_txt)
            use_cached_media: bool
                Trueの場合、media_locationsで登録された最後のメディア以降のすべてのxを扱う。T_txtはmedia_locations.shape[1]と正確に等しくある必要はない
        戻り値:
            torch.Tensor: クロスアテンションの出力
        """
        logger.info(f"MaskedCrossAttentionの順伝播開始 {x.shape=}, {media.shape=}, {None if media_locations is None else media_locations.shape=}, {use_cached_media=}")

        if not use_cached_media:
            assert (
                media_locations.shape[1] == x.shape[1]
            ), f"media_location.shape is {media_locations.shape} but x.shape is {x.shape}"

        # テキスト特徴量
        # T_txt=22
        T_txt = x.shape[1]
        logger.debug(f"{T_txt=}")

        # 画像枚数と1画像あたりの画像特徴量の数
        # T_img=3（画像枚数）
        # n=64（画像特徴量数）
        _, T_img, n = media.shape[:3]
        logger.debug(f"{T_img=}, {n=}")

        # アテンションのヘッド数
        # 8
        h = self.heads

        # レイヤー正規化を適用
        # (3, 22, 2048)
        x = self.norm(x)

        # テキスト特徴量からクエリを生成
        # アテンション計算のため次元数を削減
        # (3, 22, 2048) -> (3, 22, 512)
        q = self.to_q(x)
        logger.debug(f"{q.shape=}")

        # 画像ごとの画像特徴量をフラット化
        # (バッチサイズ, 画像枚数, 画像特徴量数, 特徴量次元) ->
        # (バッチサイズ, 画像枚数 * 画像特徴量数, 特徴量次元)
        # (3, 3, 64, 1024) -> (3, 192, 1024)
        media = rearrange(media, "b t n d -> b (t n) d")
        logger.debug(f"{media.shape=}")

        # 画像特徴量からキーとバリューを生成し、分割
        # アテンション計算のため次元数を512に削減
        # (3, 192, 1024) -> (3, 192, 512) , (3, 192, 512)
        k, v = self.to_kv(media).chunk(2, dim=-1)
        logger.debug(f"{k.shape=}, {v.shape=}")

        # クエリ、キー、バリューをヘッドに分割
        # (3, 22, 512) -> (3, 8, 22, 64)
        # (3, 192, 512) -> (3, 8, 192, 64)
        # (3, 192, 512) -> (3, 8, 192, 64)
        q, k, v = rearrange_many((q, k, v), "b n (h d) -> b h n d", h=h)
        logger.debug(f"{q.shape=}, {k.shape=}, {v.shape=}")

        # スケーリングを適用（dim_headの平方根の逆数を掛ける）
        # QKに対するスケーリングと数学的に等価
        q = q * self.scale

        # クエリとキーの内積（アテンションロジット）を取得
        # (3, 8, 22, 64) , (3, 8, 192, 64) -> (3, 8, 22, 192)
        sim = einsum("... i d, ... j d -> ... i j", q, k)
        logger.debug(f"{sim.shape=}")

        # 画像特徴量へのマスクがある場合
        # True
        if exists(media_locations):

            # 画像のインデックスを作成
            # (1, 2, 3)
            media_time = torch.arange(T_img, device=x.device) + 1
            logger.debug(f"{media_time=}")

            # False
            if use_cached_media:
                # 現在入力されたテキストトークンはキャッシュされた最後の画像を参照する
                text_time = repeat(
                    torch.count_nonzero(media_locations, dim=1),
                    "b -> b i",
                    i=T_txt,
                )
                logger.debug(f"{use_cached_media=}, {text_time=}")

            else:
                # 各テキスト特徴量が参照する画像番号を作成
                # 例: [False, False, True, False, True] -> [0, 0, 1, 1, 2]
                text_time = media_locations.cumsum(dim=-1)
                logger.debug(f"{use_cached_media=}, {text_time=}")

            # マスク操作を決定
            # equal: 直前の画像のみに注意を向ける
            mask_op = torch.eq if self.only_attend_immediate_media else torch.ge
            logger.debug(f"{mask_op=}")

            # 各テキスト特徴量が参照する画像インデックスと画像特徴量が属する画像インデックスを比較してマスクを作成
            # = 各テキスト特徴量が参照できる画像特徴量のマスク
            # text_time: (3, 22) -> (3, 1, 22, 1)
            # media_time: (3,) -> (1, 1, 1, 3 * 64)
            # (3, 1, 22, 192)
            text_to_media_mask = mask_op(
                rearrange(text_time, "b i -> b 1 i 1"),
                repeat(media_time, "j -> 1 1 1 (j n)", n=n),
            )
            logger.debug(f"{text_to_media_mask.shape=}")

            # アテンションロジットにマスクを適用（Image-Causal Modeling）
            # 参照が禁止された画像トークンに対して大きな負の値を設定
            sim = sim.masked_fill(~text_to_media_mask, -torch.finfo(sim.dtype).max)
            logger.debug(f"{sim.shape=}")

        # 数値安定性のために最大値を引く
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()

        # ソフトマックスでアテンションスコアを計算
        attn = sim.softmax(dim=-1)

        # エッジケースの処理
        # シーケンスの最初に出てくるまだどの画像も参照していないテキスト特徴量へのアテンションをゼロにする
        if exists(media_locations) and self.only_attend_immediate_media:

            # 参照する画像の無いテキスト特徴量のマスクを作成
            # (3, 22)
            text_without_media_mask = text_time == 0
            logger.debug(f"{text_without_media_mask.shape=}")

            # マスクの形状をアテンションスコアに合わせる
            # (3, 1, 22, 1)
            text_without_media_mask = rearrange(
                text_without_media_mask, "b i -> b 1 i 1"
            )
            logger.debug(f"{text_without_media_mask.shape=}")

            # アテンションスコアにマスクを適用
            # (3, 8, 22, 192)
            attn = attn.masked_fill(text_without_media_mask, 0.0)
            logger.debug(f"{attn.shape=}")

        # バリューをアテンションスコアで加重平均をとる
        # (3, 8, 22, 192) , (3, 8, 192, 64) -> (3, 8, 22, 64)
        out = einsum("... i j, ... j d -> ... i d", attn, v)
        logger.debug(f"{out.shape=}")

        # アテンションヘッドを結合
        # (3, 8, 22, 64) -> (3, 22, 512)
        out = rearrange(out, "b h n d -> b n (h d)")
        logger.debug(f"{out.shape=}")

        # 出力の線形変換を適用
        # (3, 22, 512) -> (3, 22, 2048)
        out = self.to_out(out)
        logger.debug(f"{out.shape=}")

        logger.info(f"MaskedCrossAttentionの順伝播完了 {out.shape=}")
        return out

### GatedCrossAttentionBlock

In [ ]:
class GatedCrossAttentionBlock(nn.Module):
    """
    ゲート付きクロスアテンションブロック
    論文中のGATED XATTN-DENSEブロックに対応
    MaskedCrossAttention、tan-h gating、FeedForwardで構成
    FlamingoLayerとFlamingoLMMixinで使用
    """

    def __init__(
        self,
        *,
        dim,
        dim_visual,
        dim_head=64,
        heads=8,
        ff_mult=4,
        only_attend_immediate_media=True,
    ):
        logger.info(f"GatedCrossAttentionBlockを初期化 {dim=}, {dim_visual=}, {dim_head=}, {heads=}, {ff_mult=}, {only_attend_immediate_media=}")

        super().__init__()

        # クロスアテンションモジュールの初期化
        self.attn = MaskedCrossAttention(
            dim=dim, # 2048
            dim_visual=dim_visual, # 1024
            dim_head=dim_head, # 64
            heads=heads, # 8
            only_attend_immediate_media=only_attend_immediate_media, # True
        )

        # クロスアテンションの直後につけるtanh-gatingのパラメータ
        self.attn_gate = nn.Parameter(torch.tensor([0.0]))

        # フィードフォワードネットワークの初期化
        self.ff = FeedForward(dim, mult=ff_mult)

        # フィードフォワードの直後につけるtanh-gatingのパラメータ
        self.ff_gate = nn.Parameter(torch.tensor([0.0]))

        logger.info("GatedCrossAttentionBlockの初期化完了")

    def forward(
        self,
        x,
        media,
        media_locations=None,
        use_cached_media=False,
    ):
        """
        ゲート付きクロスアテンションブロックの順伝播

        引数:
            x (torch.Tensor): テキスト特徴量
                shape (B, T_txt, D_txt)
            media (torch.Tensor): 画像特徴量
                shape (B, T_img, n, D_img) ここでnは潜在変数の次元
            media_locations: x内のメディアトークンを識別するブールマスク
                shape (B, T_txt)
            use_cached_media: bool
                Trueの場合、media_locationsで登録された最後のメディア以降のすべてのxを扱う。T_txtはmedia_locations.shape[1]と正確に等しくある必要はない

        戻り値:
            torch.Tensor: ゲート付きクロスアテンションブロックの出力
        """
        logger.info(f"GatedCrossAttentionBlockの順伝播開始 {x.shape=}, {x.dtype=} {media.shape=}, {None if media_locations is None else media_locations.shape=}, {use_cached_media=}")
    
        # 前半のクロスアテンションとtanh-gatingと残差接続
        # (3, 22, 2048) -> (3, 22, 2048)
        x = (
            self.attn(
                x,
                media,
                media_locations=media_locations,
                use_cached_media=use_cached_media,
            )
            * self.attn_gate.tanh()
            + x
        )

        # 後半のフィードフォワードとtanh-gatingと残差接続
        # (3, 22, 2048) -> (3, 22, 2048)
        x = self.ff(x) * self.ff_gate.tanh() + x

        logger.info(f"GatedCrossAttentionBlockの順伝播完了 {x.shape=} {x.dtype=}")
        return x

### PerceiverAttention

In [ ]:
class PerceiverAttention(nn.Module):
    """
    Perceiver Attentionモジュール
    学習可能な潜在クエリとCLIPが出力した画像特徴量のキーとバリューに基づいてクロスアテンションを計算する
    PerceiverResamplerの一部
    """

    def __init__(self, *, dim, dim_head=64, heads=8):
        logger.info(f"PerceiverAttentionを初期化 {dim=}, {dim_head=}, {heads=}")

        super().__init__()

        # アテンションロジットQKのスケーリング係数
        # 64^-0.5 = 0.125
        self.scale = dim_head**-0.5
        logger.debug(f"{self.scale=}")

        # アテンションのヘッド数
        # 8
        self.heads = heads

        # アテンションの内部次元
        # 64 * 8 = 512
        inner_dim = dim_head * heads
        logger.debug(f"{inner_dim=}")

        # 画像特徴量に適応するレイヤー正規化層
        # 1024
        self.norm_media = nn.LayerNorm(dim)

        # 潜在クエリに適応するレイヤー正規化層
        # 1024
        self.norm_latents = nn.LayerNorm(dim)

        # 潜在クエリからアテンション入力のクエリに変換する線形射影層
        # 1024 -> 512
        self.to_q = nn.Linear(dim, inner_dim, bias=False)

        # 画像特徴量からアテンション入力のキーとバリューに変換する線形射影層
        # 1024 -> 1024
        self.to_kv = nn.Linear(dim, inner_dim * 2, bias=False)

        # アテンション出力への線形変換層
        # 512 -> 1024
        self.to_out = nn.Linear(inner_dim, dim, bias=False)

        logger.info("PerceiverAttentionの初期化完了")

    def forward(self, x, latents):
        """
        Perceiver Attentionの順伝播
        学習可能な潜在クエリと画像特徴量のキーとバリューに基づいてアテンションを計算する

        引数:
            x (torch.Tensor): 画像特徴量
                shape (b, T, n1, D)
            latents (torch.Tensor): 潜在クエリ
                shape (b, T, n2, D)
                shape (b, T, n2, D)

        戻り値:
            torch.Tensor: Perceiver Attentionの出力
        """
        logger.info(f"PerceiverAttentionの順伝播開始 {x.shape=}, {x.dtype=} {latents.shape=}, {latents.dtype=}")

        # 画像特徴量にレイヤー正規化層を適用
        # (3, 3, 256, 1024)
        x = self.norm_media(x)

        # 潜在クエリにレイヤー正規化層を適用
        # (3, 3, 64, 1024)
        latents = self.norm_latents(latents)

        # アテンションのヘッド数
        # 8
        h = self.heads

        # クエリを生成
        # (3, 3, 64, 1024) -> (3, 3, 64, 512)
        q = self.to_q(latents)
        logger.debug(f"{q.shape=}")

        # キーとバリューを同時に生成
        # (3, 3, 256, 1024) と (3, 3, 64, 1024) を連結 -> (3, 3, 320, 1024)
        kv_input = torch.cat((x, latents), dim=-2)
        logger.debug(f"{kv_input.shape=}")

        # キーとバリューを分割
        # (3, 3, 320, 1024) -> (3, 3, 320, 512) と (3, 3, 320, 512)
        k, v = self.to_kv(kv_input).chunk(2, dim=-1)
        logger.debug(f"{k.shape=}, {v.shape=}")

        # クエリ、キー、バリューをヘッドに分割
        # (3, 3, 64, 512) -> (3, 8, 3, 64, 64)
        # (3, 3, 320, 512) -> (3, 8, 3, 320, 64)
        # (3, 3, 320, 512) -> (3, 8, 3, 320, 64)
        q, k, v = rearrange_many((q, k, v), "b t n (h d) -> b h t n d", h=h)
        logger.debug(f"{q.shape=}, {k.shape=}, {v.shape=}")

        # スケーリングを適用（dim_headの平方根の逆数を掛ける）
        q = q * self.scale

        # クエリとキーの内積（アテンションロジット）を取得
        # (3, 8, 3, 64, 64) と (3, 8, 3, 320, 64) -> (3, 8, 3, 64, 320)
        sim = einsum("... i d, ... j d  -> ... i j", q, k)

        # 数値安定性のために最大値を引く
        # (3, 8, 3, 64, 320)
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()

        # ソフトマックスでアテンションスコアを計算
        # (3, 8, 3, 64, 320)
        attn = sim.softmax(dim=-1)

        # バリューをアテンションスコアで加重平均
        # (3, 8, 3, 64, 320) と (3, 8, 3, 320, 64) -> (3, 8, 3, 64, 64)
        out = einsum("... i j, ... j d -> ... i d", attn, v)

        # ヘッドを結合
        # (3, 8, 3, 64, 64) -> (3, 3, 64, 512)
        out = rearrange(out, "b h t n d -> b t n (h d)", h=h)

        # 出力の線形変換を適用
        # (3, 3, 64, 512) -> (3, 3, 64, 1024)
        out = self.to_out(out)

        logger.info(f"PerceiverAttentionの順伝播完了 {out.shape=} {out.dtype=}")
        return out

### PerceiverResampler

In [ ]:
class PerceiverResampler(nn.Module):
    """
    Perceiver Resamplerモジュール
    6層のPerceiver Attentionとフィードフォワードブロックで構成される
    Flamingoモジュールで使用
    """

    def __init__(
        self,
        *,
        dim,
        depth=6,
        dim_head=64,
        heads=8,
        num_latents=64,
        max_num_media=None,
        max_num_frames=None,
        ff_mult=4,
    ):
        logger.info(f"PerceiverResamplerを初期化 {dim=}, {depth=}, {dim_head=}, {heads=}, {num_latents=}, {max_num_media=}, {max_num_frames=}, {ff_mult=}")

        super().__init__()

        # 学習可能な潜在クエリを初期化
        # (64, 1024)
        self.latents = nn.Parameter(torch.randn(num_latents, dim))

        # 動画の時間フレームを区別するための学習可能な位置埋め込みを初期化
        # None
        self.frame_embs = (
            nn.Parameter(torch.randn(max_num_frames, dim))
            if exists(max_num_frames)
            else None
        )

        # 複数画像の順序を区別するための学習可能な位置埋め込みを初期化
        # None
        self.media_time_embs = (
            nn.Parameter(torch.randn(max_num_media, 1, dim))
            if exists(max_num_media)
            else None
        )

        # Perceiver Attentionとフィードフォワードのブロックを6層積み重ねる
        self.layers = nn.ModuleList([])
        for _ in range(depth): # 6
            self.layers.append(
                nn.ModuleList(
                    [
                        PerceiverAttention(dim=dim, dim_head=dim_head, heads=heads),
                        FeedForward(dim=dim, mult=ff_mult),
                    ]
                )
            )

        # 最終的なレイヤー正規化層
        self.norm = nn.LayerNorm(dim)

        logger.info("PerceiverResamplerの初期化完了")

    def forward(self, x):
        """
        Perceiver Resamplerの順伝播

        引数:
            x (torch.Tensor): 画像特徴量
                shape (b, T, F, v, D)
                b: バッチサイズ
                T: 画像の枚数
                F: フレーム数 (1)
                v: 画像特徴量数 (256)
                D: 特徴量の次元数 (1024)
        戻り値:
            torch.Tensor: Perceiver Resamplerの出力
                shape (b, T, n, D) where n is self.num_latents
        """
        logger.info(f"PerceiverResamplerの順伝播開始 {x.shape=}, {x.dtype=}")

        # 1) 画像特徴量の準備

        # バッチサイズ, 画像の枚数, フレーム数, 画像特徴量数
        # 3, 3, 1, 256
        b, T, F, v = x.shape[:4]

        # 動画の場合
        # False
        if exists(self.frame_embs):
            # フレーム位置埋め込みを複製
            frame_embs = repeat(self.frame_embs[:F], "F d -> b T F v d", b=b, T=T, v=v)
            logger.debug(f"{frame_embs.shape=}")

            # 位置埋め込みを特徴量に加算
            x = x + frame_embs

        # 長い特徴量シーケンスに平坦化
        # (バッチサイズ, 画像の枚数, フレーム数, 視覚トークン数, 特徴量次元) ->
        # (バッチサイズ, 画像の枚数, フレーム数 * 視覚トークン数, 特徴量次元)
        # (3, 3, 1, 256, 1024) -> (3, 3, 256, 1024)
        x = rearrange(
            x, "b T F v d -> b T (F v) d"
        )
        logger.debug(f"{x.shape=}")

        # 複数画像の場合
        # False
        if exists(self.media_time_embs):
            # 画像時間位置埋め込みを加算
            x = x + self.media_time_embs[:T]
            logger.debug(f"{x.shape=}")

        # 2) 潜在クエリの準備

        # 潜在クエリをバッチサイズと画像枚数分複製
        # (64, 1024) -> (3, 3, 64, 1024)
        latents = repeat(self.latents, "n d -> b T n d", b=b, T=T)
        logger.debug(f"{latents.shape=}")

        # 3) Perceiver Resamplerの適用

        # Perceiver Attentionとフィードフォワードのブロックを順伝播
        # 6回繰り返す
        for attn, ff in self.layers:
            # Perceiver Attentionを適用し、残差接続
            latents = attn(x, latents) + latents
            # フィードフォワードを適用し、残差接続
            latents = ff(latents) + latents

        # 最終的なレイヤー正規化層を適用
        # (3, 3, 64, 1024)
        out = self.norm(latents)

        logger.info(f"PerceiverResamplerの順伝播完了 {out.shape=} {out.dtype=}")
        return out

### FlamingoLayer

In [ ]:
class FlamingoLayer(nn.Module):
    """
    Flamingoレイヤー
    LM layer（DecoderLayer）とGATED XATTN-DENSE（GatedCrossAttentionBlock）のラッパー
    FlamingoMixinによって、言語モデルの元のデコーダーレイヤーがこのレイヤーで置き換えられる
    """

    def __init__(
        self, gated_cross_attn_layer, decoder_layer, gradient_checkpointing=False
    ):
        logger.info(f"FlamingoLayerを初期化 {gradient_checkpointing=}")

        super().__init__()

        # GATED XATTN-DENSE
        self.gated_cross_attn_layer = gated_cross_attn_layer

        # LM layer
        self.decoder_layer = decoder_layer

        # PerceiverResamplerで計算した視覚特徴量を格納する属性
        self.vis_x = None

        # テキスト内の<image>トークンの位置を示すアテンションマスクを格納する属性
        self.media_locations = None

        # GATED XATTN-DENSEの勾配チェックポイントの設定
        # 勾配チェックポイントは計算コストの高い結果のみをキャッシュしメモリ使用量を抑える技術
        if self.gated_cross_attn_layer is not None:
            # False
            self.gated_cross_attn_layer._use_gradient_checkpointing = (
                gradient_checkpointing
            )

        # LM layerの勾配チェックポイントの設定
        # False
        self.decoder_layer._use_gradient_checkpointing = gradient_checkpointing

        logger.info("FlamingoLayerの初期化完了")

    def is_conditioned(self) -> bool:
        return self.vis_x is not None and self.media_locations is not None

    # Used this great idea from this implementation of Flamingo (https://github.com/dhansmair/flamingo-mini/)
    def condition_vis_x(self, vis_x):
        self.vis_x = vis_x

    def condition_media_locations(self, media_locations):
        self.media_locations = media_locations

    def condition_use_cached_media(self, use_cached_media):
        self.use_cached_media = use_cached_media

    def forward(
        self,
        lang_x,
        attention_mask=None,
        **decoder_layer_kwargs,
    ):
        """
        Flamingoレイヤーの順伝播
        視覚特徴量を参照して、言語特徴量を処理する

        引数:
            lang_x (torch.Tensor): 言語特徴量
                shape (B, T_txt, D_txt)
            attention_mask (torch.Tensor, optional): アテンションマスク。デフォルトはNone。
            **decoder_layer_kwargs: LM layerの順伝播に渡される追加のキーワード引数
        """
        logger.info(f"FlamingoLayerの順伝播開始 {lang_x.shape=}, {lang_x.dtype=}, {None if attention_mask is None else attention_mask.shape=}, {None if decoder_layer_kwargs["past_key_value"] is None else decoder_layer_kwargs["past_key_value"].shape=}, {None if decoder_layer_kwargs["attn_bias"] is None else decoder_layer_kwargs["attn_bias"].shape=}, {decoder_layer_kwargs["is_causal"]=}")

        # 1) GATED XATTN-DENSEを適用（FlamingoのクロスアテンションTransformerデコーダー）

        # GATED XATTN-DENSEレイヤーが存在する場合
        if self.gated_cross_attn_layer is not None:

            if self.vis_x is None:
                raise ValueError("vis_x must be conditioned before forward pass")

            if self.media_locations is None:
                raise ValueError(
                    "media_locations must be conditioned before forward pass"
                )

            # 視覚情報を参照して言語特徴量を処理（クロスアテンション）
            # (3, 22, 2048) -> (3, 22, 2048)
            lang_x = self.gated_cross_attn_layer(
                lang_x, # (3, 22, 2048)
                self.vis_x, # (3, 3, 64, 1024)
                media_locations=self.media_locations,
                use_cached_media=self.use_cached_media,
            )
            logger.debug(f"{lang_x.shape=}, {lang_x.dtype=}")

        # 2) LM Layerを適用（元の言語モデルのセルフアテンションTransformerデコーダー）

        # (3, 22, 2048) -> (3, 22, 2048)
        lang_x = self.decoder_layer(
            lang_x, # (3, 22, 2048)
            attention_mask=attention_mask, # None
            **decoder_layer_kwargs, # past_key_value, attn_bias, is_causal
        )

        logger.info(f"FlamingoLayerの順伝播終了 {lang_x[0].shape=} {lang_x[1]=}")

        return lang_x

### FlamingoLMMixin

In [ ]:
class FlamingoLMMixin(nn.Module):
    """
    既存の言語モデルにGATED XATTN-DENSEを追加してFlamingo化するミックスインクラス
    """

    def set_decoder_layers_attr_name(self, decoder_layers_attr_name):
        self.decoder_layers_attr_name = decoder_layers_attr_name

    def _get_decoder_layers(self):
        return getattr_recursive(self, self.decoder_layers_attr_name)

    def _set_decoder_layers(self, value):
        setattr_recursive(self, self.decoder_layers_attr_name, value)

    def init_flamingo(
        self,
        media_token_id,
        lang_hidden_size,
        vis_hidden_size,
        cross_attn_every_n_layers,
        gradient_checkpointing,
    ):
        """
        既存の言語モデルにGATED XATTN-DENSEを追加してFlamingo化するミックスインクラスを初期化する
        またメディアトークンID（<image>のトークンID）を保存して、メディア位置を計算する

        引数:
            media_token_id (int): <image>のトークンID
            lang_hidden_size (int): 言語モデルの隠れ状態の次元数
            vis_hidden_size (int): ビジュアル特徴量の次元数
            cross_attn_every_n_layers (int): トランスフォーマーレイヤーの後にクロスアテンションを適用する頻度
            gradient_checkpointing (bool): 勾配チェックポイントを使用するかどうか
        戻り値:
            None
        """
        logger.info(f"FlamingoLMMixinを初期化 {media_token_id=}, {lang_hidden_size=}, {vis_hidden_size=}, {cross_attn_every_n_layers=}, {gradient_checkpointing=}")

        # 元のデコーダーレイヤーを保存
        self.old_decoder_blocks = self._get_decoder_layers()

        # 言語モデルのN層ごとにGATED XATTN-DENSEレイヤーを作成
        # 1層ごとに作成
        self.gated_cross_attn_layers = nn.ModuleList(
            [
                GatedCrossAttentionBlock(
                    dim=lang_hidden_size, # 2048
                    dim_visual=vis_hidden_size # 1024
                )
                if (layer_idx + 1) % cross_attn_every_n_layers == 0
                else None
                for layer_idx, _ in enumerate(self._get_decoder_layers())
            ]
        )

        # 既存の言語モデルのデコーダーレイヤーをFlamingoLayerでラップする
        self.init_flamingo_layers(gradient_checkpointing)

        # メディアトークンID（<image>トークンのID）を保存
        self.media_token_id = media_token_id

        # Flamingoレイヤーが初期化されたことを示すフラグを有効化
        self.initialized_flamingo = True

        # キャッシュされた視覚情報を使用するかどうかのフラグを初期化
        self._use_cached_vision_x = False

        logger.info("FlamingoLMMixinの初期化完了")

    def init_flamingo_layers(self, gradient_checkpointing):
        """
        既存の言語モデルのデコーダーレイヤーをFlamingoLayerでラップする
        init_flamingoで呼び出される

        引数:
            gradient_checkpointing (bool): 勾配チェックポイントを使用するかどうか
        """
        logger.info(f"複数のFlamingoLayerを初期化 {gradient_checkpointing=}")

        # 既存の言語モデルのデコーダーレイヤーをFlamingoLayerでラップする
        # 24層
        self._set_decoder_layers(
            nn.ModuleList(
                [
                    FlamingoLayer(
                        gated_cross_attn_layer, decoder_layer, gradient_checkpointing
                    )
                    for gated_cross_attn_layer, decoder_layer in zip(
                        self.gated_cross_attn_layers, self.old_decoder_blocks
                    )
                ]
            )
        )
        logger.info("複数のFlamingoLayerの初期化完了")

    def forward(self, input_ids, attention_mask, **kwargs):
        """
        既存の言語モデルが言語特徴量を処理する前に実行される初期化関数
        """
        logger.info(f"FlamingoLMMixinの順伝播開始 {input_ids.shape=}, {input_ids.dtype=}, {None if attention_mask is None else attention_mask.shape=}, {kwargs.keys()}")

        if not self.initialized_flamingo:
            raise ValueError(
                "Flamingo layers are not initialized. Please call `init_flamingo` first."
            )

        # 1) GATED XATTN-DENSEの設定（言語モデル全体の条件付け）

        # <image>トークンの位置を特定
        media_locations = input_ids == self.media_token_id
        logger.debug(f"{media_locations.shape=}, {media_locations.dtype=}")

        # キャッシュを使用するかどうかを決定
        # _use_cached_vision_xがTrueで、処理中のトークンに<image>トークンが含まれていないのみ
        # 新しい画像が来ていない場合、最後に見た画像を参照する
        # (3, 23)
        use_cached_media_locations = (
            self._use_cached_vision_x
            and self.is_conditioned()
            and not media_locations.any()
        )
        logger.debug(f"{use_cached_media_locations=}")

        # すべてのFlamingoLayerに設定を反映
        for layer in self._get_decoder_layers():
            if not use_cached_media_locations:
                # 言語特徴量が参照する画像特徴量の位置を設定
                layer.condition_media_locations(media_locations)

            # キャッシュを使用するかどうかを設定
            layer.condition_use_cached_media(use_cached_media_locations)

        # FlamingoLayerを追加後の言語モデルの順伝播を実行
        kwargs["input_ids"] = input_ids
        kwargs["attention_mask"] = attention_mask
        res = super().forward(**kwargs)

        logger.info("FlamingoLMMixinの順伝播完了")
        return res

    def is_conditioned(self) -> bool:
        """Check whether all decoder layers are already conditioned."""
        return all(l.is_conditioned() for l in self._get_decoder_layers())

    def clear_conditioned_layers(self):
        """
        順伝播や生成の後に実行されるクリア処理
        """

        for layer in self._get_decoder_layers():

            # 視覚トークンをクリア
            layer.condition_vis_x(None)

            # メディア位置をクリア
            layer.condition_media_locations(None)

            # キャッシュ使用フラグをクリア
            layer.condition_use_cached_media(None)

### Flamingo

In [ ]:
class Flamingo(nn.Module):
    """
    Flamingoモデル
    ビジョンエンコーダーと言語エンコーダーを組み合わせたマルチモーダルモデル
    """

    def __init__(
        self,
        vision_encoder: nn.Module,
        lang_encoder: nn.Module,
        eoc_token_id: int,
        media_token_id: int,
        vis_dim: int,
        cross_attn_every_n_layers: int = 1,
        gradient_checkpointing: bool = False,
    ):
        """
        Flamingoモデルの初期化

        引数:
            vision_encoder (nn.Module): Hugging FaceのCLIPModel
            lang_encoder (nn.Module): Hugging Faceの因果言語モデル
            eoc_token_id (int): <|endofchunk|>のトークンID
            media_token_id (int): <image>のトークンID
            vis_dim (int): 視覚特徴量の次元数
            cross_attn_every_n_layers (int, optional): クロスアテンションを適用する頻度（デフォルトは1）
            gradient_checkpointing (bool, optional): 勾配チェックポイントを使用するかどうか。デフォルトはFalse。
        """
        logger.info(f"Flamingoモデルを初期化 {eoc_token_id=}, {media_token_id=}, {vis_dim=}, {cross_attn_every_n_layers=}, {gradient_checkpointing=}")

        super().__init__()

        # EOCトークンID（<|endofchunk|>）を設定
        # 50277
        self.eoc_token_id = eoc_token_id

        # メディアトークンID（<image>）を設定
        # 50278
        self.media_token_id = media_token_id

        # 視覚特徴量の次元数
        # 1024
        self.vis_dim = vis_dim

        # 言語特徴量の次元数
        # 2048
        if hasattr(lang_encoder.config, "d_model"):
            self.lang_dim = lang_encoder.config.d_model  # mpt uses d_model
        else:
            self.lang_dim = lang_encoder.config.hidden_size
        logger.info(f"{self.lang_dim=}")

        # 視覚エンコーダーを初期化（CLIP）
        self.vision_encoder = vision_encoder.visual

        # PerceiverResamplerを初期化
        self.perceiver = PerceiverResampler(dim=self.vis_dim)

        # 言語エンコーダーを初期化
        self.lang_encoder = lang_encoder

        # 言語モデルにGATED XATTN-DENSEレイヤーを挿入してFlamingo化
        self.lang_encoder.init_flamingo(
            media_token_id=media_token_id,
            lang_hidden_size=self.lang_dim,
            vis_hidden_size=self.vis_dim,
            cross_attn_every_n_layers=cross_attn_every_n_layers,
            gradient_checkpointing=gradient_checkpointing,
        )

        # 勾配チェックポイントの設定
        self._use_gradient_checkpointing = gradient_checkpointing

        # PerceiverResamplerの勾配チェックポイントの設定
        self.perceiver._use_gradient_checkpointing = gradient_checkpointing

        logger.info("Flamingoモデルの初期化完了")

    def forward(
        self,
        vision_x: torch.Tensor,
        lang_x: torch.Tensor,
        attention_mask: torch.Tensor = None,
        labels: torch.Tensor = None,
        clear_conditioned_layers: bool = True,
        past_key_values=None,
        use_cache: bool = False,
    ):
        """
        今回は使わない
        Flamingoの順伝播

        引数:
            vision_x (torch.Tensor): ビジョン入力
                shape (B, T_img, F, C, H, W) with F=1
            lang_x (torch.Tensor): 言語入力ID
                shape (B, T_txt)
            attention_mask (torch.Tensor, optional): アテンションマスク。デフォルトはNone。
            labels (torch.Tensor, optional): ラベル。デフォルトはNone。
            clear_conditioned_layers: Trueの場合、順伝播が完了したら条件付きレイヤーをクリアします。
                同じセットの画像が別の順伝播で再利用される場合は、これをfalseに設定します。
            past_key_values: 事前計算された値を言語モデルに渡します。
                Hugging FaceのCausalLMモデルのpast_key_valuesドキュメントを参照してください。
            use_cache: キャッシュされたキー値を使用するかどうか。Hugging Face CausalLMモデルのuse_cacheドキュメントを参照してください。
        戻り値:
            CausalLMOutputWithPast: 言語モデルの出力
        """
        logger.info(f"Flamingoの順伝播開始 {vision_x.shape=}, {lang_x.shape=}, {None if attention_mask is None else attention_mask.shape=}, {None if labels is None else labels.shape=}, {clear_conditioned_layers=}, {use_cache=}")

        # 1) 入力の検証

        assert (
            self.lang_encoder.initialized_flamingo
        ), "Flamingo layers are not initialized. Please call `init_flamingo` first."

        assert (
            self.lang_encoder._use_cached_vision_x or vision_x is not None
        ), "Must provide either vision_x or have precached media using cache_media()."

        # 2) 生の画像テンソルのエンコードとFlamingoLayerの条件付け

        if self.lang_encoder._use_cached_vision_x:
            # Case: use cached; vision_x should be cached and other
            # vision-related inputs should not be provided.
            assert (
                vision_x is None
            ), "Expect vision_x to be None when media has been cached using cache_media(). Try uncache_media() first."
            assert self.lang_encoder.is_conditioned()

        else:
            # 生の画像テンソルを視覚トークンにエンコードし、Flamingoレイヤーを条件づけ
            self._encode_vision_x(vision_x=vision_x)

            # <image>トークンの位置を特定し、アテンションマスクを作成
            self._condition_media_locations(input_ids=lang_x)

        # 3) 条件付けした言語モデルで順伝播

        output = self.lang_encoder(
            input_ids=lang_x,
            attention_mask=attention_mask,
            labels=labels,
            past_key_values=past_key_values,
            use_cache=use_cache,
        )
        logger.debug(f"{output=}")

        # 4) Flamingoレイヤーの条件づけをクリア

        if clear_conditioned_layers:
            self.lang_encoder.clear_conditioned_layers()

        logger.info(f"Flamingoの順伝播完了")

        return output

    def generate(
        self,
        vision_x: torch.Tensor,
        lang_x: torch.Tensor,
        attention_mask: torch.Tensor = None,
        **kwargs,
    ):
        """
        画像とテキストプロンプトからテキストを生成する

        引数:
            vision_x (torch.Tensor): ビジョン入力
                shape (B, T_img, F, C, H, W)
                同じチャンク内の画像はT_imgに沿って連結され、フレームはFに沿って連結されます
                現在、F=1のみがサポートされています（単一フレームのビデオ）
            lang_x (torch.Tensor): 言語入力
                shape (B, T_txt)
            **kwargs: Hugging Face CausalLMモデルのgenerateドキュメントを参照してください。注目すべきkwargs:
                max_length (int, optional): 出力の最大長。デフォルトはNone。
                attention_mask (torch.Tensor, optional): アテンションマスク。デフォルトはNone。
                num_beams (int, optional): ビームの数。デフォルトは1。
                max_new_tokens (int, optional): 最大新規トークン数。デフォルトはNone。
                temperature (float, optional): 温度。デフォルトは1.0。
                top_k (int, optional): トップk。デフォルトは50。
                top_p (float, optional): トップp。デフォルトは1.0。
                no_repeat_ngram_size (int, optional): 繰り返し禁止ngramサイズ。デフォルトは0。
                length_penalty (float, optional): 長さペナルティ。デフォルトは1.0。
                num_return_sequences (int, optional): 返されるシーケンスの数。デフォルトは1。
                do_sample (bool, optional): サンプリングを行うかどうか。デフォルトはFalse。
                early_stopping (bool, optional): 早期停止。デフォルトはFalse。
        戻り値:
            torch.Tensor: 生成されたトークンが追加されたlang_x
        """
        logger.info(f"Flamingoのgenerate開始 {vision_x.shape=}, {lang_x.shape=}, {None if attention_mask is None else attention_mask.shape=}, {kwargs.keys()}")

        # 1) 初期化

        num_beams = kwargs.pop("num_beams", 1)

        # 3
        if num_beams > 1:
            # ビームサーチ用に複製
            # (batch_size, T_img, F, C, H, W) -> (batch_size * num_beams, T_img, F, C, H, W)
            # (1, 3, 1, 3, 224, 224) -> (3, 3, 1, 3, 224, 224)
            vision_x = vision_x.repeat_interleave(num_beams, dim=0)
            logger.debug(f"repeated {vision_x.shape=}, {lang_x.shape=}")

        # 2) 生の画像テンソルからFlamingoLayerを条件づけ 

        self.lang_encoder._use_cached_vision_x = True

        # 生の画像テンソルをフラット化し、6層のPerceiverAttentionで順伝播し、視覚トークンにエンコード
        # Flamingoレイヤーを条件づけ
        # (3, 3, 1, 3, 224, 224) -> (3, 3, 64, 1024)
        self._encode_vision_x(vision_x=vision_x)

        # 3) 条件付けした言語モデルでテキストを生成

        eos_token_id = kwargs.pop("eos_token_id", self.eoc_token_id)

        # 自己回帰的にテキストを生成
        output = self.lang_encoder.generate(
            input_ids=lang_x,
            attention_mask=attention_mask,
            eos_token_id=eos_token_id,
            num_beams=num_beams,
            **kwargs,
        )
        logger.debug(f"{output.shape=}")

        # 4) Flamingoレイヤーの条件づけをクリア

        self.lang_encoder.clear_conditioned_layers()

        self.lang_encoder._use_cached_vision_x = False

        logger.info(f"Flamingoのgenerate完了 {output.shape=} {output.dtype=}")
        return output

    def _encode_vision_x(self, vision_x: torch.Tensor):
        """
        PerceiverResamplerで生の画像テンソルを視覚トークンにエンコードし、Flamingoレイヤーを条件づけする

        引数:
            vision_x (torch.Tensor): 生の画像テンソル
                shape (B, T_img, F, C, H, W)
                同一チャンク内の画像はT_imgに沿って連結され、フレームはFに沿って連結される
                現在、F=1のみがサポート（単一フレームのビデオ）
        戻り値:
            None

        実装はflamingo-miniのコードを参考に再構成
        https://github.com/dhansmair/flamingo-mini
        """
        logger.info(f"生の画像テンソルのエンコード開始 {vision_x.shape=}, {vision_x.dtype=}")

        # 1) 入力の検証

        # (3, 3, 1, 3, 224, 224)
        assert vision_x.ndim == 6, "vision_x should be of shape (b, T_img, F, C, H, W)"

        # バッチサイズ, 画像枚数, フレーム数（1のみサポート）
        # 3, 3, 1
        b, T, F = vision_x.shape[:3]

        assert F == 1, "Only single frame supported"

        # 2) バッチの平坦化

        # (バッチサイズ, 画像枚数, フレーム数, チャネル数, 高さ, 幅) ->
        # ((バッチサイズ * 画像枚数 * フレーム数), チャネル数, 高さ, 幅)
        # (3, 3, 1, 3, 224, 224) -> (9, 3, 224, 224)
        vision_x = rearrange(vision_x, "b T F c h w -> (b T F) c h w")
        logger.debug(f"flatten {vision_x.shape=}")

        # 3) 視覚エンコーダー（CLIP）で生の画像テンソルをエンコード

        with torch.no_grad():
            # (9, 3, 224, 224) -> (9, 256, 1024)
            vision_x = self.vision_encoder(vision_x)[1]
            logger.debug(f"encoded {vision_x.shape=}, {vision_x.dtype=}")

        # 4) 形状を戻す

        # (9, 256, 1024) -> (3, 3, 1, 256, 1024)
        vision_x = rearrange(vision_x, "(b T F) v d -> b T F v d", b=b, T=T, F=F)
        logger.debug(f"unflatten {vision_x.shape=}")

        # 5) PerceiverResamplerで6層のPerceiverAttentionを順伝播し、固定長の視覚トークンに変換

        # (3, 3, 1, 256, 1024) -> (3, 3, 64, 1024)
        vision_x = self.perceiver(vision_x)
        logger.debug(f"resampled {vision_x.shape=}, {vision_x.dtype=}")

        # 6) 言語モデルのFlamingoレイヤーを条件づけ

        for layer in self.lang_encoder._get_decoder_layers():
            layer.condition_vis_x(vision_x)

        logger.info(f"生の画像テンソルのエンコード完了 {vision_x.shape=} {vision_x.dtype=}")

    def wrap_fsdp(self, wrapper_kwargs, device_id):
        """
        分散学習用

        Manually wraps submodules for FSDP and move other parameters to device_id.

        Why manually wrap?
        - all parameters within the FSDP wrapper must have the same requires_grad.
            We have a mix of frozen and unfrozen parameters.
        - model.vision_encoder.visual needs to be individually wrapped or encode_vision_x errors
            See: https://github.com/pytorch/pytorch/issues/82461#issuecomment-1269136344

        The rough wrapping structure is:
        - FlamingoModel
            - FSDP(FSDP(vision_encoder))
            - FSDP(FSDP(perceiver))
            - lang_encoder
                - FSDP(FSDP(input_embeddings))
                - FlamingoLayers
                    - FSDP(FSDP(gated_cross_attn_layer))
                    - FSDP(FSDP(decoder_layer))
                - FSDP(FSDP(output_embeddings))
                - other parameters

        Known issues:
        - Our FSDP strategy is not compatible with tied embeddings. If the LM embeddings are tied,
            train with DDP or set the --freeze_lm_embeddings flag to true.
        - With FSDP + gradient ckpting, one can increase the batch size with seemingly no upper bound.
            Although the training curves look okay, we found that downstream performance dramatically
            degrades if the batch size is unreasonably large (e.g., 100 MMC4 batch size for OPT-125M).

        FAQs about our FSDP wrapping strategy:
        Why double wrap?
        As of torch==2.0.1, FSDP's _post_forward_hook and _post_backward_hook
        only free gathered parameters if the module is NOT FSDP root.

        Why unfreeze the decoder_layers?
        See https://github.com/pytorch/pytorch/issues/95805
        As of torch==2.0.1, FSDP's _post_backward_hook is only registed if the flat param
        requires_grad=True. We need the postback to fire to avoid OOM.
        To effectively freeze the decoder layers, we exclude them from the optimizer.

        What is assumed to be frozen v. unfrozen?
        We assume that the model is being trained under normal Flamingo settings
        with these lines being called in factory.py:
            ```
            # Freeze all parameters
            model.requires_grad_(False)
            assert sum(p.numel() for p in model.parameters() if p.requires_grad) == 0

            # Unfreeze perceiver, gated_cross_attn_layers, and LM input embeddings
            model.perceiver.requires_grad_(True)
            model.lang_encoder.gated_cross_attn_layers.requires_grad_(True)
            [optional] model.lang_encoder.get_input_embeddings().requires_grad_(True)
            ```
        """
        # unfreeze the decoder layers
        for block in self.lang_encoder.old_decoder_blocks:
            block.requires_grad_(True)

        # wrap in FSDP
        with enable_wrap(wrapper_cls=FSDP, **wrapper_kwargs):
            self.perceiver = wrap(wrap(self.perceiver))
            self.lang_encoder.old_decoder_blocks = nn.ModuleList(
                wrap(wrap(block)) for block in self.lang_encoder.old_decoder_blocks
            )
            self.lang_encoder.gated_cross_attn_layers = nn.ModuleList(
                wrap(wrap(layer)) if layer is not None else None
                for layer in self.lang_encoder.gated_cross_attn_layers
            )
            self.lang_encoder.init_flamingo_layers(self._use_gradient_checkpointing)
            self.lang_encoder.set_input_embeddings(
                wrap(wrap(self.lang_encoder.get_input_embeddings()))
            )
            self.lang_encoder.set_output_embeddings(
                wrap(wrap(self.lang_encoder.get_output_embeddings()))
            )
            self.vision_encoder = wrap(wrap(self.vision_encoder))  # frozen

        # manually move non-FSDP managed parameters to device_id
        # these are all in lang_encoder
        apply_with_stopping_condition(
            module=self.lang_encoder,
            apply_fn=lambda m: m.to(device_id),
            apply_condition=lambda m: len(list(m.children())) == 0,
            stopping_condition=lambda m: isinstance(m, FSDP),
        )

        # exclude the original decoder layers from the optimizer
        for block in self.lang_encoder.old_decoder_blocks:
            for p in block.parameters():
                p.exclude_from_optimizer = True

        # set up clip_grad_norm_ function
        def clip_grad_norm_(max_norm):
            self.perceiver.clip_grad_norm_(max_norm)
            for layer in self.lang_encoder.gated_cross_attn_layers:
                if layer is not None:
                    layer.clip_grad_norm_(max_norm)
            self.lang_encoder.get_input_embeddings().clip_grad_norm_(max_norm)

        self.clip_grad_norm_ = clip_grad_norm_

    def _condition_media_locations(self, input_ids: torch.Tensor):
        """
        Compute the media token locations from lang_x and condition the language model on these.
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
        """
        media_locations = input_ids == self.media_token_id

        for layer in self.lang_encoder._get_decoder_layers():
            layer.condition_media_locations(media_locations)

    def cache_media(self, input_ids: torch.Tensor, vision_x: torch.Tensor):
        """
        Pre-cache a prompt/sequence of images / text for log-likelihood evaluations.
        All subsequent calls to forward() will generate attending to the LAST
        image in vision_x.
        This is not meant to be used to cache things for generate().
        Args:
            input_ids (torch.Tensor): Language input
                shape (B, T_txt)
            vision_x (torch.Tensor): Vision input
                shape (B, T_img, F, C, H, W)
                Images in the same chunk are collated along T_img, and frames are collated along F
                Currently only F=1 is supported (single-frame videos)
        """
        self._encode_vision_x(vision_x=vision_x)
        self._condition_media_locations(input_ids=input_ids)
        self.lang_encoder._use_cached_vision_x = True

    def uncache_media(self):
        """
        Clear all conditioning.
        """
        self.lang_encoder.clear_conditioned_layers()
        self.lang_encoder._use_cached_vision_x = False

### 推論

In [ ]:
clip_vision_encoder_path="ViT-L-14"
clip_vision_encoder_pretrained="openai"
lang_encoder_path="anas-awadalla/mpt-1b-redpajama-200b"
tokenizer_path="anas-awadalla/mpt-1b-redpajama-200b"
cross_attn_every_n_layers=1
cache_dir=None
use_local_files = False
decoder_layers_attr_name: str = None
freeze_lm_embeddings: bool = False

In [ ]:
# 訓練済みのCLIPの視覚エンコーダーと画像プロセッサーを読み込み

# CLIP
vision_encoder, _, image_processor = open_clip.create_model_and_transforms(
    clip_vision_encoder_path,
    pretrained=clip_vision_encoder_pretrained,
    cache_dir=cache_dir,
)

# 視覚エンコーダーが視覚特徴量を出力するように設定
vision_encoder.visual.output_tokens = True

In [ ]:
# テキストトークナイザーを読み込み

# GPTNeoXTokenizerFast
# https://huggingface.co/docs/transformers/en/model_doc/gpt_neox
text_tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path,
    local_files_only=use_local_files,
    trust_remote_code=True,
    cache_dir=cache_dir,
)

# Flamingoの特殊トークンをトークナイザーに追加
text_tokenizer.add_special_tokens(
    {"additional_special_tokens": ["<|endofchunk|>", "<image>"]}
)

if text_tokenizer.pad_token is None:
    # GPT2モデルはパッドトークンを持たない
    # 損失のためにラベルを修正するために使用する
    text_tokenizer.add_special_tokens({"pad_token": "<PAD>"})
    logger.info("パッドトークンを追加")

logger.info(f"トークナイザーを読み込み {text_tokenizer}")

In [ ]:
# 訓練済みの言語エンコーダーを読み込み

# MosaicGPT: 24層、2048次元、16ヘッドのMPTモデルをRedPajama-200Bデータセットで事前訓練
# MPT-1B-RedPajama-200B
# MPTは、Mosaic Pretrained Transformerの略でMosaicMLが開発元
# https://huggingface.co/anas-awadalla/mpt-1b-redpajama-200b
lang_encoder = AutoModelForCausalLM.from_pretrained(
    lang_encoder_path,
    local_files_only=use_local_files,
    trust_remote_code=True,
    cache_dir=cache_dir,
)
logger.info(f"テキストエンコーダーを読み込み完了 {lang_encoder=} {lang_encoder.config=}")

In [ ]:
def extend_instance(obj, mixin):
    """
    クラスインスタンスにミックスインを適用する
    引数:
        obj: クラスインスタンス
        mixin: ミックスインクラス
    """

    # クラスを取得
    base_cls = obj.__class__

    # クラス名を取得
    base_cls_name = obj.__class__.__name__

    # 新しいクラスを作成
    obj.__class__ = type(
        base_cls_name,
        (mixin, base_cls), # 左のクラスが優先され、forward()はmixinのものが使われる
        {}
    )

In [ ]:
# True
if "mpt-1b-redpajama-200b" in lang_encoder_path:

    # 入力埋め込み層を取得・設定するクラスをミックスイン

    class EmbeddingFnMixin:
        def get_input_embeddings(self):
            return self.transformer.wte

        def set_input_embeddings(self, new_embeddings):
            self.transformer.wte = new_embeddings

    extend_instance(lang_encoder, EmbeddingFnMixin)
    logger.info("MPT EmbeddingFnMixinを適用")
   
# 言語モデルをFlamingo言語モデルに変換
# これによりFlamingoLMMixinのforward()が優先される
extend_instance(lang_encoder, FlamingoLMMixin)
logger.info("FlamingoLMMixinを適用")

In [ ]:
# 既存の言語モデルのデコーダーレイヤーの属性名を推測または設定

# 既知のモデルとそのデコーダーレイヤーの属性名のマッピング
__KNOWN_DECODER_LAYERS_ATTR_NAMES = {
    "opt": "model.decoder.layers",
    "gptj": "transformer.h",
    "gpt-j": "transformer.h",
    "pythia": "gpt_neox.layers",
    "llama": "model.layers",
    "gptneoxforcausallm": "gpt_neox.layers",
    "mpt": "transformer.blocks",
    "mosaicgpt": "transformer.blocks",
}

def _infer_decoder_layers_attr_name(model):
    """
    既知のモデルクラス名に基づいて、言語モデルのデコーダーレイヤーの属性名を推測する
    引数:
        model: 言語モデルのnn.Moduleインスタンス
    戻り値:
        str: デコーダーレイヤーの属性名
    """

    # 対応する属性名を見つけて返す
    for k in __KNOWN_DECODER_LAYERS_ATTR_NAMES:
        if k.lower() in model.__class__.__name__.lower():
            return __KNOWN_DECODER_LAYERS_ATTR_NAMES[k]

    raise ValueError(
        f"We require the attribute name for the nn.ModuleList in the decoder storing the transformer block layers. Please supply this string manually."
    )

if decoder_layers_attr_name is None:
    # transformer.blocks
    decoder_layers_attr_name = _infer_decoder_layers_attr_name(lang_encoder)
    logger.info(f"デコーダーレイヤーの属性名を推測 {decoder_layers_attr_name=}")

# Flamingoが言語モデルのデコーダーレイヤーを見つけられるように設定
lang_encoder.set_decoder_layers_attr_name(decoder_layers_attr_name)
logger.info(f"デコーダーレイヤーの属性名を設定 {decoder_layers_attr_name=}")

# 50,280
lang_encoder.resize_token_embeddings(len(text_tokenizer))
logger.info(f"テキストエンコーダーのトークン埋め込みをリサイズ {len(text_tokenizer)=}")

In [ ]:
# Flamingoモデルを初期化

model = Flamingo(
    vision_encoder,
    lang_encoder,
    text_tokenizer.encode("<|endofchunk|>")[-1],
    text_tokenizer.encode("<image>")[-1],
    vis_dim=open_clip.get_model_config(clip_vision_encoder_path)["vision_cfg"][
        "width"
    ],
    cross_attn_every_n_layers=cross_attn_every_n_layers,
)

# 2,559,117,360 = 2.56B
num_params = sum(p.numel() for p in model.parameters())
logger.info(f"Flamingoモデル総パラメータ数: {num_params:,}")

In [ ]:
# 推論時は必要ない

model.requires_grad_(False)
assert sum(p.numel() for p in model.parameters() if p.requires_grad) == 0
logger.info("モデルの全パラメータをフリーズ")

model.perceiver.requires_grad_(True)
logger.info("Perceiverのパラメータをアンフリーズ")

model.lang_encoder.gated_cross_attn_layers.requires_grad_(True)
logger.info("Gated Cross Attention Layersのパラメータをアンフリーズ")

# True
if not freeze_lm_embeddings:
    model.lang_encoder.get_input_embeddings().requires_grad_(True)
    logger.info("LM Input Embeddingsのパラメータをアンフリーズ")

num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
logger.info(f"訓練可能なパラメータ数: {num_trainable_params:,}")

In [ ]:
# 訓練済みのFlamingoモデルのチェックポイントを読み込み

# PerceiverResampler, Gated XATTN-DENSE, LM Input Embeddingsが含まれる
# https://huggingface.co/openflamingo/OpenFlamingo-3B-vitl-mpt1b
checkpoint_path = hf_hub_download("openflamingo/OpenFlamingo-3B-vitl-mpt1b", "checkpoint.pt")
model.load_state_dict(torch.load(checkpoint_path), strict=False)

In [ ]:
demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)
demo_image_one

In [ ]:
demo_image_two = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
        stream=True
    ).raw
)
demo_image_two

In [ ]:
query_image = Image.open(
    requests.get(
        "http://images.cocodataset.org/test-stuff2017/000000028352.jpg", 
        stream=True
    ).raw
)
query_image

In [ ]:
# 画像を前処理してテンソルに変換

# image_processorで3枚の画像を前処理し、リストに格納
# [(1, 3, 224, 224), (1, 3, 224, 224), (1, 3, 224, 224)]
vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
logger.info(f"画像を前処理 {vision_x[0].shape=}, {vision_x[0].dtype=}")

# 画像を1つのテンソルに結合
# (num_media, channels, height, width)
# (3, 3, 224, 224)
vision_x = torch.cat(vision_x, dim=0)
logger.info(f"画像を結合 {vision_x.shape=}, {vision_x.dtype=}")

# 画像テンソルをリシェイプしてFlamingoモデルに適合させる
# (batch_size, num_media, num_frames, channels, height, width)
# (3, 3, 224, 224) -> (1, 3, 1, 3, 224, 224)
vision_x = vision_x.unsqueeze(1).unsqueeze(0)
logger.info(f"画像をリシェイプ {vision_x.shape=}, {vision_x.dtype=}")

In [ ]:
# テキストを前処理

# 生成のためにパディングを左側に設定
text_tokenizer.padding_side = "left"

# <image>と<|endofchunk|>トークンを含むテキストのリストを作成
text_list = ["<image>An image of two cats.<|endofchunk|><image>An image of a bathroom sink.<|endofchunk|><image>An image of"]
logger.info(f"テキストを準備 {text_list=}")

# トークン化
lang_x = text_tokenizer(text_list, return_tensors="pt",)
logger.info(f"テキストを前処理 {lang_x['input_ids']=}")

In [ ]:
# テキスト生成
generated_text = model.generate(
    vision_x=vision_x, # 生の画像テンソル
    lang_x=lang_x["input_ids"], # テキストトークン
    attention_mask=lang_x["attention_mask"], # アテンションマスク
    max_new_tokens=20, # 生成する最大新規トークン数
    num_beams=3, # ビームサーチのビーム数
)
logger.info(f"テキスト生成完了 {generated_text}")

In [ ]:
decoded = text_tokenizer.decode(generated_text[0])
logger.info(f"生成されたテキスト: {decoded}")